In [10]:
import pandas as pd

df_tags = pd.read_csv('../data/raw/ml-25m/tags.csv')
df_gscores = pd.read_csv('../data/raw/ml-25m/genome-scores.csv')
df_gtags = pd.read_csv('../data/raw/ml-25m/genome-tags.csv')
df_links = pd.read_csv('../data/raw/ml-25m/links.csv')
df_movies = pd.read_csv('../data/raw/ml-25m/movies.csv')
df_ratings = pd.read_csv('../data/raw/ml-25m/ratings.csv')

In [11]:
# Create combined dataframe from the smaller dataframes and cleaning it
df = pd.merge(df_movies, df_gscores, how="left", on='movieId')
df_gscores = pd.merge(df, df_gtags, how="left", on='tagId')
df_gscores.drop(columns=['tagId'], inplace=True)

In [13]:
# Keep only the top 40 tags for each movie
df_gscores = df_gscores.sort_values(by=['movieId', 'relevance'], ascending=[True, False])
df_gscores = df_gscores.groupby('movieId', group_keys=False).apply(lambda x: x.nlargest(40, 'relevance')).reset_index(drop=True)

C:\Users\Leo\AppData\Local\Temp\ipykernel_19856\3995359352.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_gscores = df_gscores.groupby('movieId', group_keys=False).apply(lambda x: x.nlargest(40, 'relevance')).reset_index(drop=True)


In [14]:
#The same movieId refers to the same movie between `ratings.csv`, `tags.csv`, `movies.csv`, and `links.csv`
df = pd.merge(df_movies, df_ratings, how="left", on=['movieId'])
df = pd.merge(df, df_tags, on=['movieId', 'userId', 'timestamp'], how='left')
df = pd.merge(df, df_gtags, on=['tag'], how='left')
df = pd.merge(df, df_gscores, on=['tagId', 'movieId'], how='left')

In [15]:
# Merge and clean more
df_ratings.drop(columns=['timestamp'], inplace=True)
df = pd.merge(df_gscores, df_ratings, on='movieId', how='left')
df.dropna(inplace=True)
df['userId'] = df['userId'].astype(int)
# removed user that is outlyer
df = df[df['userId'] != 75309]

In [17]:
# Count the occurrences of each movieId and filter out movieIds that occur less than 2000 times
movie_counts = df['movieId'].value_counts()
movie_ids_to_keep = movie_counts[movie_counts >= 2000].index
df = df[df['movieId'].isin(movie_ids_to_keep)]


In [19]:
##### Deadlines:
- Report 1. Data viz (5 graphs) + preprocessing. Friday, May 17
- Report 2. Modelling. Friday, June 14
- Final report. Friday, June 21


C:\Users\Leo\AppData\Local\Temp\ipykernel_19856\4005301193.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('movieId', group_keys=False).apply(sample_log_base_1_004)


movieId
318      2833
356      2833
296      2827
593      2809
2571     2804
         ... 
8966     1905
86298    1905
3388     1905
6890     1904
4291     1904
Name: count, Length: 2427, dtype: int64

In [21]:
# Safe the dataframe locally

df.to_csv('../data/processed/preprocessed_data_movielens.csv', index=False)

In [23]:
# test if the outlyer is still in the dataframe
user_exists = df['userId'].isin([75309]).any()
print("User 75390 exists in DataFrame:", user_exists)

User 75390 exists in DataFrame: False
